In [102]:
import requests
import csv
import pandas as pd

In [ ]:
GITHUB_TOKEN = 'Your Token'
headers = {
    'Authorization': f'Bearer {GITHUB_TOKEN}',
    'Accept': 'application/vnd.github.v3+json',
}
search_url = "https://api.github.com/search/users"
params = {
    "q": "location:Beijing followers:>500",
    "per_page": 100,
    "page": 1
}

In [160]:
column = ["login", "name", "company", "location", "email", "hireable", "bio", "public_repos", "followers", "following", "created_at"]
with open("users.csv", "w", newline="") as file:
    writer = csv.DictWriter(file, fieldnames=column)
    writer.writeheader()

repo_column = ['login', 'full_name', 'created_at', 'stargazers_count', 'watchers_count', 'language', 'has_projects', 'has_wiki', 'license_name']
with open("repositories.csv", "w", newline="") as file:
    writer = csv.DictWriter(file, fieldnames=repo_column)
    writer.writeheader()

def write_repo_data(r_data):
    with open("repositories.csv","a", encoding="utf-8", newline="") as file:
        writer = csv.DictWriter(file, fieldnames=repo_column)
        for repo in r_data:
            repo_log = {
                        'login':repo['owner']['login'],
                        'full_name':repo['full_name'],
                        'created_at':repo['created_at'],
                        'stargazers_count':repo['stargazers_count'],
                        'watchers_count':repo['watchers_count'],
                        'language':repo['language'],
                        'has_projects':repo['has_projects'],
                        'has_wiki':repo['has_wiki'],
                        'license_name':repo['license']['key'] if repo['license'] else "",
                    }
            writer.writerow(repo_log)
        print('Repo Data Created',len(r_data))
        
def write_data(data):
    with open("users.csv", "a", encoding="utf-8", newline="") as file:
        writer = csv.DictWriter(file, fieldnames=column)
        user_log = {
                    "login": data["login"],
                    "name": data["name"],
                    "company": data["company"].strip().lstrip("@").upper() if data["company"] else "",
                    "location": data["location"],
                    "email": data["email"],
                    "hireable": data["hireable"],
                    "bio": data["bio"],
                    "public_repos": data["public_repos"],
                    "followers": data["followers"],
                    "following": data["following"],
                    "created_at": data["created_at"]
                }
        if user_log['bio']:
            user_log['bio'] = user_log['bio'].replace('\r\n', '') 
            user_log['bio'] = user_log['bio'].strip()
        writer.writerow(user_log)

In [161]:
def over_users(data,p,h):
    count=1

    for each in data:
        repo_param = {
                'sort':'created',
                'per_page':100,
                'page':0   
            }
        user_url = each['url']
        user = requests.get(user_url, headers=h)
        user = user.json()
        

        write_data(user)
        print('user data Created', count)

        repo_url = each['repos_url']
        for p in range(1,6):
            repo_param['page']+=1
            repos = requests.get(repo_url,headers=h, params=repo_param)
            repos = repos.json()
            if repos == []:
                break

            write_repo_data(repos)
        
        count+=1

In [162]:
def fetch_users(s_url,p,h):
    params['page']=1
    while True:
        get_users = requests.get(s_url, params=p, headers=h)
        get_users = get_users.json()
        
        if get_users['items'] == []:
            print('Data')
            break
        #function call
        over_users(get_users['items'],p,h)
        params['page']+=1

In [163]:
fetch_users(search_url,params, headers)

user data Created 1
Repo Data Created 99
user data Created 2
Repo Data Created 89
user data Created 3
Repo Data Created 54
user data Created 4
Repo Data Created 50
user data Created 5
Repo Data Created 100
Repo Data Created 2
user data Created 6
Repo Data Created 24
user data Created 7
Repo Data Created 86
user data Created 8
Repo Data Created 100
Repo Data Created 100
Repo Data Created 100
Repo Data Created 100
Repo Data Created 100
user data Created 9
Repo Data Created 100
Repo Data Created 45
user data Created 10
Repo Data Created 28
user data Created 11
Repo Data Created 81
user data Created 12
Repo Data Created 26
user data Created 13
Repo Data Created 43
user data Created 14
Repo Data Created 48
user data Created 15
Repo Data Created 73
user data Created 16
Repo Data Created 100
Repo Data Created 2
user data Created 17
Repo Data Created 100
Repo Data Created 16
user data Created 18
Repo Data Created 6
user data Created 19
Repo Data Created 59
user data Created 20
Repo Data Create

In [84]:
df = pd.read_csv('users.csv',)
df

,login,name,company,location,email,hireable,bio,public_repos,followers,following,created_at
0,michaelliao,Crypto Michael,NaN,"Beijing, China",askxuefeng@gmail.com,NaN,Crypto developer.,99,37242,3,2010-11-06T12:21:35Z
1,daimajia,代码家,ZHENFUND BEIJING,"Beijing, China",daimajia@gmail.com,NaN,Zhenfund VP of Investment.,89,24636,271,2012-10-07T02:40:06Z
2,xiaolai,xiaolai,INBLOCKCHAIN,beijing,lixiaolai@gmail.com,NaN,A lifelong student.,54,19250,37,2009-11-13T18:29:42Z
3,draveness,Draven,SPECTRA-FUND,"Beijing, China",i@draven.co,True,HFT / C++ / Go,50,13010,28,2014-01-24T16:22:01Z
4,hongyangAndroid,张鸿洋,WANANDROID.COM,"Beijing,China",623565791@qq.com,True,学习ing,102,12986,35,2015-01-26T07:05:45Z
...,...,...,...,...,...,...,...,...,...,...,...
356,yu-changqian,Changqian,HUAZHONG UNIVERSITY OF SCIENCE AND TECHNOLOGY,Beijing,NaN,True,Researcher for CV/VLM | Senior Development Eng...,18,507,199,2013-12-08T17:09:43Z
357,zhangzichao,Zichao Zhang,NaN,"Beijing, China",zhangzichao17@gmail.com,NaN,NaN,22,505,181,2012-02-16T12:13:42Z
358,fuxuemingzhu,负雪明烛,BUPT,"Beijing,China",NaN,NaN,算法题，数据挖掘，推荐系统，大数据。,34,505,91,2014-08-29T07:41:07Z
359,tcc0lin,tcc0lin,MT,BEIJING CHINA,hanqiulun1123@gmail.com,True,Focus On Android Security,33,502,61,2019-01-21T14:40:53Z


In [164]:
df[df['public_repos']>=500]

,login,name,company,location,email,hireable,bio,public_repos,followers,following,created_at
7,i5ting,狼叔,ALIBABA,china beijing,i5ting@126.com,True,focus on node & rust & web3。2023 Yak Shaving R...,902,9074,1127,2012-12-24T23:28:15Z
116,xiaoweiruby,肖威（xiaowei）,才华横溢科技（北京）有限责任公司,"beijing,China",shenzhoudance@gmail.com,True,才华横溢科技创始人兼首席执行官。创建 SuperxSchool.com 。微信 xiaowe...,860,1267,16119,2015-11-03T07:25:34Z
180,LinuxSuRen,Rick,OPENSOURCE-F2F,"Beijing, China",NaN,True,Dev and a host of open-source Podcasts @openso...,1842,926,2,2012-02-19T06:28:06Z
283,hailiang-wang,Hai Liang W.,CHATOPERA,"Beijing, China",NaN,NaN,Chatopera 创始人，《智能问答与深度学习》核心作者，为企业交付智能客服系统、智能对话...,1056,609,189,2013-02-12T04:40:57Z
315,wanglong,HackerVirus,NaN,BEIJING,nmgimust_wl@163.com,NaN,"Keep An Open Mind, Strive For Excellence。",610,555,61,2010-06-28T02:57:26Z


In [183]:
df[df['public_repos']>=500]['public_repos'].sum()

5270

In [181]:
df[df['public_repos']<500]['public_repos'].sum()

27083

In [182]:
df['public_repos'].sum()

32353